
<br>
==================================================<br>
Comparing edge-based and region-based segmentation<br>
==================================================<br>
In this example, we will see how to segment objects from a background. We use<br>
the ``coins`` image from ``skimage.data``, which shows several coins outlined<br>
against a darker background.<br>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from skimage import data
from skimage.exposure import histogram

In [ ]:
coins = data.coins()
hist, hist_centers = histogram(coins)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3))
axes[0].imshow(coins, cmap=plt.cm.gray)
axes[0].axis('off')
axes[1].plot(hist_centers, hist, lw=2)
axes[1].set_title('histogram of gray values')

####################################################################<br>
<br>
Thresholding<br>
============<br>
<br>
A simple way to segment the coins is to choose a threshold based on the<br>
histogram of gray values. Unfortunately, thresholding this image gives a<br>
binary image that either misses significant parts of the coins or merges<br>
parts of the background with the coins:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3), sharey=True)

In [ ]:
axes[0].imshow(coins > 100, cmap=plt.cm.gray)
axes[0].set_title('coins > 100')

In [ ]:
axes[1].imshow(coins > 150, cmap=plt.cm.gray)
axes[1].set_title('coins > 150')

In [ ]:
for a in axes:
    a.axis('off')

In [ ]:
plt.tight_layout()

####################################################################<br>
Edge-based segmentation<br>
=======================<br>
<br>
Next, we try to delineate the contours of the coins using edge-based<br>
segmentation. To do this, we first get the edges of features using the<br>
Canny edge-detector.

In [ ]:
from skimage.feature import canny

In [ ]:
edges = canny(coins)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))
ax.imshow(edges, cmap=plt.cm.gray)
ax.set_title('Canny detector')
ax.axis('off')

####################################################################<br>
These contours are then filled using mathematical morphology.

In [ ]:
from scipy import ndimage as ndi

In [ ]:
fill_coins = ndi.binary_fill_holes(edges)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))
ax.imshow(fill_coins, cmap=plt.cm.gray)
ax.set_title('filling the holes')
ax.axis('off')

####################################################################<br>
Small spurious objects are easily removed by setting a minimum size for<br>
valid objects.

In [ ]:
from skimage import morphology

In [ ]:
coins_cleaned = morphology.remove_small_objects(fill_coins, 21)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))
ax.imshow(coins_cleaned, cmap=plt.cm.gray)
ax.set_title('removing small objects')
ax.axis('off')

####################################################################<br>
However, this method is not very robust, since contours that are not<br>
perfectly closed are not filled correctly, as is the case for one unfilled<br>
coin above.<br>
<br>
Region-based segmentation<br>
=========================<br>
<br>
We therefore try a region-based method using the watershed transform.<br>
First, we find an elevation map using the Sobel gradient of the image.

In [ ]:
from skimage.filters import sobel

In [ ]:
elevation_map = sobel(coins)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))
ax.imshow(elevation_map, cmap=plt.cm.gray)
ax.set_title('elevation map')
ax.axis('off')

####################################################################<br>
Next we find markers of the background and the coins based on the extreme<br>
parts of the histogram of gray values.

In [ ]:
markers = np.zeros_like(coins)
markers[coins < 30] = 1
markers[coins > 150] = 2

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))
ax.imshow(markers, cmap=plt.cm.nipy_spectral)
ax.set_title('markers')
ax.axis('off')

####################################################################<br>
Finally, we use the watershed transform to fill regions of the elevation<br>
map starting from the markers determined above:

In [ ]:
from skimage import segmentation

In [ ]:
segmentation_coins = segmentation.watershed(elevation_map, markers)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))
ax.imshow(segmentation_coins, cmap=plt.cm.gray)
ax.set_title('segmentation')
ax.axis('off')

####################################################################<br>
This last method works even better, and the coins can be segmented and<br>
labeled individually.

In [ ]:
from skimage.color import label2rgb

In [ ]:
segmentation_coins = ndi.binary_fill_holes(segmentation_coins - 1)
labeled_coins, _ = ndi.label(segmentation_coins)
image_label_overlay = label2rgb(labeled_coins, image=coins, bg_label=0)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3), sharey=True)
axes[0].imshow(coins, cmap=plt.cm.gray)
axes[0].contour(segmentation_coins, [0.5], linewidths=1.2, colors='y')
axes[1].imshow(image_label_overlay)

In [ ]:
for a in axes:
    a.axis('off')

In [ ]:
plt.tight_layout()

In [ ]:
plt.show()